In [1]:
# Importando bibliotecas
import os
from time import sleep
import csv
import import_ipynb
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from msedge.selenium_tools import Edge, EdgeOptions
from credencial import username, password # Dados do login
from leitura_google_sheets import ano_pae3, processos_pae3 # variáveis para uso nas funções
from leitura_google_sheets import processos_pae4 # variáveis para uso nas funções

importing Jupyter notebook from leitura_google_sheets.ipynb
leitura do google sheets realizada com sucesso!


**INÍCIO DO SCRIPT PARA PESQUISA DOS PROCESSOS NO PAE 3.0**

In [2]:
# Crie uma instância do driver do Microsoft Edge
driver = webdriver.Edge()
driver.get('https://www.sistemas.pa.gov.br/governodigital/public/main/index.xhtml')
driver.maximize_window()

In [3]:
# Login no sistema
wait = WebDriverWait(driver, 20)
# Encontrar e preencher o campo de usuário
click_usuario = wait.until(
    EC.presence_of_element_located((By.XPATH,'//*[@id="form_login:login_username"]')))
click_usuario.send_keys(username)
# Encontrar e preencher o campo de senha
click_senha = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="form_login:login_password"]')))
click_senha.send_keys(password)
# Clicar no botão de login
botao_login = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="form_login:button_login"]')))
botao_login.click()

In [4]:
# Acesso PAE3
# Armazene o identificador da janela atual
current_window = driver.current_window_handle
# Clique no elemento que abre a nova aba ou janela
click_pae3 = driver.find_element(
    By.XPATH,'//*[@id="form_sistema:submit_area"]/div/div[3]/div[1]/a/img')
click_pae3.click()
# Aguarde a nova aba ou janela ser aberta
wait.until(EC.number_of_windows_to_be(2))
# Mudar para a nova aba ou janela
for window_handle in driver.window_handles:
    if window_handle != current_window:
        driver.switch_to.window(window_handle)
        break

In [5]:
# Navegação dos menus até pesquisa de processos
# Clique no menu 'Protocolo Eletrônico'
proc_adm_ele = driver.find_element(
    By.XPATH, '//*[@id="iconmenu_vert:panelMenuGroupProtocoloEletronico"]')
proc_adm_ele.click()
# Aguarde até que o submenu 'Entrada e Saída' esteja visível e clicável
entrada_saida = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="iconmenu_vert:j_id52"]')))
entrada_saida.click()
# Aguarde até que o submenu 'Localizar Protocolo Eletrônico' esteja visível e clicável
localizar_protocolo = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="iconmenu_vert:panelLocalizarProtocoloEletronico"]')))
localizar_protocolo.click()

In [6]:
# Limpando o ano do espaço a ser utilizado para iniciar as pesquisas dos processos
# Aguarde até que o campo de entrada de ano esteja visível e clicável
entrada_ano = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="protocolo_consult_params:ano_protocolo"]')))
# Limpe o campo de entrada de ano
entrada_ano.clear()

In [8]:
# Iniciando as interações no PAE a fim de saber a localização dos processos
local_processos3 = []

def check_procesos(ano, processo):
    for a, p in zip(ano, processo):
        wait = WebDriverWait(driver, 10)
        entrada_ano = wait.until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="protocolo_consult_params:ano_protocolo"]')))
        
        sleep(1)
        entrada_ano.clear()
        entrada_ano.send_keys(a)
        sleep(2)

        click_processo = driver.find_element(
            By.XPATH, '//*[@id="protocolo_consult_params:sequencial_protocolo"]')
        click_processo.clear()
        click_processo.send_keys(p)
        sleep(2)

        click_pesquisa_processo = driver.find_element(
            By.XPATH, '//*[@id="protocolo_consult_params:botaoPesquisar"]')
        click_pesquisa_processo.click()

        # Aguardar até que a tabela com o resultado seja visível
        wait = WebDriverWait(driver, 20)
        setor_processo = wait.until(
            EC.presence_of_element_located((By.XPATH, '//*[contains(@id, "j_id609:table")]')))

        setor = setor_processo.text
        local_processos3.append(setor)

        # Navegação dos menus até a pesquisa de processos para reiniciar a página atual
        proc_adm_ele = driver.find_element(
            By.XPATH, '//*[@id="iconmenu_vert:panelMenuGroupProtocoloEletronico"]').click()
        sleep(1)
        entrada_saida = driver.find_element(
            By.XPATH, '//*[@id="iconmenu_vert:j_id52"]').click()
        sleep(1)
        entrada_saida = driver.find_element(
            By.XPATH, '//*[@id="iconmenu_vert:panelLocalizarProtocoloEletronico"]').click()
        sleep(3)
        
# Chamando as variáveis dos processos que serão utilizadas no loop de repetição
ano = ano_pae3
processo = processos_pae3
# Chamando a função
check_procesos(ano, processo)
# Fechar o navegador após a conclusão
driver.quit()

In [9]:
# Tratamento para retirar apenas o setor onde os processos se encontram
localizacao = []

for setor in local_processos3: localizacao.append(setor.split('»')[-2].strip().split('-')[0])

In [10]:
# Realizando a junção do ano com o porcesso
processos_pae3_ = [f'{ano}/{processo}' for ano, processo in zip(ano_pae3, processos_pae3)]

In [11]:
'''Criação de um dicionário com os dados para construção de um dataframe com 
dados do processo PAE3 e a localização atualizada'''

data_proc_pae3 = {'Nº DO PROCESSO': processos_pae3_, 'LOCALIZAÇÃO': localizacao}
tabela_localizacao_pae3 = pd.DataFrame(data_proc_pae3)

**INÍCIO DO SCRIPT PARA PESQUISA DOS PROCESSOS NO PAE 4.0**

In [17]:
# Crie uma instância do driver do Microsoft Edge
driver = webdriver.Edge()
# Abra o navegador
driver.get('https://www.sistemas.pa.gov.br/governodigital/public/main/index.xhtml')
driver.maximize_window()

In [18]:
# Login no sistema
wait = WebDriverWait(driver, 20)
# Encontrar e preencher o campo de usuário
click_usuario = wait.until(
    EC.presence_of_element_located((By.XPATH,'//*[@id="form_login:login_username"]')))
click_usuario.send_keys(username)  # Dados do login estão no script credencial.py
# Encontrar e preencher o campo de senha
click_senha = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="form_login:login_password"]')))
click_senha.send_keys(password)  # Dados do login estão no script credencial.py
# Clicar no botão de login
botao_login = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="form_login:button_login"]')))
botao_login.click()

In [19]:
# Acesso ao PAE4
# Armazene o identificador da janela atual
current_window = driver.current_window_handle
# Clique no elemento que abre a nova aba ou janela
click_pae3 = driver.find_element(
    By.XPATH,'//*[@id="form_sistema:submit_area"]/div/div[2]/div[1]/a/img').click()
# Aguarde a nova aba ou janela ser aberta
wait.until(EC.number_of_windows_to_be(2))
# Mudar para a nova aba ou janela
for window_handle in driver.window_handles:
    if window_handle != current_window:
        driver.switch_to.window(window_handle)
        break

In [20]:
# Navegação dos menus até pesquisa de processos
wait = WebDriverWait(driver, 30)
# Aguardar até que o elemento "Tramitados" seja clicável
proc_tramit = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//div[@class="q-item__label"][contains(normalize-space(text()), "Tramitados")]')))
driver.execute_script("arguments[0].click();", proc_tramit)
# Aguardar até que o filtro "Filtrar (0 filtros ativos)" seja clicável
filtro = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//span[@class="block"][text()="Filtrar (0 filtros ativos)"]')))
driver.execute_script("arguments[0].click();", filtro)
# Aguardar até que o campo "Número do protocolo" seja clicável
add_num_proc = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//input[@aria-label="Número do protocolo"]')))
driver.execute_script("arguments[0].click();", add_num_proc)

In [21]:
# Iniciando as interações no PAE a fim de saber a localização dos processos
local_processos4 = []

def verificar_procesos(processos=list):
        
    for x in processos:     
        wait = WebDriverWait(driver, 20)
        
        add_num_proc = driver.find_element(
            By.XPATH,'//input[@aria-label="Número do protocolo"]')
        sleep(2)  
            
        add_num_proc.send_keys(x)
        sleep(1)
        click_pesquisa_proc = driver.find_element(
            By.XPATH, '//span[@class="block"][text()= "Aplicar Filtros"]')      
        click_pesquisa_proc.click()
        wait = WebDriverWait(driver, 10)
        setor_processo = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//div[contains(normalize-space(text()), "Setor Atual:")]')))
        
        setor = setor_processo.text
        local_processos4.append(setor)
        sleep(1)
        
        clicar_filtro = driver.find_element(
            By.XPATH, '//*[@id="q-app"]/div/div/div[3]/div/div[1]/div/main/div[2]/div/div/div[2]/button/span[2]/span/span').click()
        sleep(1)
        limpar_filtro = driver.find_element(
            By.XPATH, '//span[@class="block"][text()= "Limpar Filtros"]').click()
        sleep(1)
        
# Chamando a função
verificar_procesos(processos_pae4)
# finalizado pae4
driver.quit()

In [22]:
# Tratamento para retirar apenas o setor onde os processos se encontram
localizacao_pae4 = []

for setor in local_processos4:
    localizacao_pae4.append(setor.split(' - ')[-2].strip().split('-')[0])

In [23]:
# Realizando a junção do 'E-' ao numero do processo
processos_pae4 = ['E-' + ano for ano in processos_pae4]

In [24]:
'''Criação de um dicionário com os dados para construção de um dataframe com 
dados do processo PAE4 e a localização atualizada'''

data_proc_pae4 = {'Nº DO PROCESSO': processos_pae4, 'LOCALIZAÇÃO': localizacao_pae4}
tabela_localizacao_pae4 = pd.DataFrame(data_proc_pae4)

,Nº DO PROCESSO,LOCALIZAÇÃO
0,E-2022/2004008,DAF
1,E-2022/2002731,CORI
2,E-2023/2026174,SCOM
3,E-2023/2122216,CJUR
4,E-2023/2001044,NCIN
5,E-2023/2028918,CORI
6,E-2023/2046962,SCOM
7,E-2023/2100684,NCIN
8,E-2023/2120254,COFI
9,E-2023/2142920,DAF


**FINALIZAÇÃO DO PROCESSO DE ATUALIZAÇÃO DOS LOCAIS DOS PROCESSOS NO SISTEM PAE**

In [25]:
# Concatenar os dataframes PAE3 e PAE4
relacao_proc_localizacao_df = pd.concat(
    [tabela_localizacao_pae3, tabela_localizacao_pae4], ignore_index=True, sort=False)

In [27]:
# Salvar dataframe no formato excel
relacao_proc_localizacao_df.to_excel('planilha_localizacao.xlsx', index=False)


In [28]:
print('Verificação dos processos no PAE realizada com sucesso!')

Verificação dos processos no PAE realizada com sucesso!
